## 从头搭建决策树

riversxiao


今天准备尝试写一篇学习笔记--搭建一个决策树模型

前两天和公司询问公司前辈，得知公司大部分模型都是基于树模型。基于之前对树模型的认知，以及使用sklearn的体验，我觉得很有必要更加深入的去了解这个模型，于是今天，我就从头来搭建一个决策树模型吧

当然。。。。

学习还是从模仿开始，这篇文章大部分的框架是从大神[Josh Gordon](https://github.com/random-forests)这里搬运过来的，此处应有emoji，不过第一版我就先照搬大神的代码了

![膜拜](http://t.biaoqing888.com/uploads/allimg/170323/149023645472812342.jpg)

好啦，膜拜结束，开始动手

In [3]:
# 代码是py3写的
# 创建一个小数据集
# 格式：每一行都是一个样本
# 最后一列是标签
# 前两行是特征
# 这个数据集主要是拿来用在公司的一个教学上面的
# 数据集是一个分辨垃圾邮箱和正常邮箱的数据集，哈哈，邮箱可是精华啊
# 第一列是邮箱域名
# 第二列是邮箱名字混乱程度 1->低 3 ->高

training_data = [
    ['谷歌', 3, '垃圾'],
    ['网易', 2, '正常'],
    ['雅虎', 3, '垃圾'],
    ['苹果', 1, '正常'],
    ['网易', 2, '垃圾'],
    ['网易', 1, '正常'],
    ['雅虎', 2, '垃圾'],
    ['苹果', 2, '正常'],
]


In [4]:
# 列名
header = ["域名", "混乱", "标签"]

In [5]:
def unique_vals(rows, col):
    '''找到每一列的特异值'''
    return set([row[col] for row in rows])

In [6]:
####
#示例
unique_vals(training_data, 0)
####

{'网易', '苹果', '谷歌', '雅虎'}

In [7]:
def class_counts(rows):
    """统计正常邮件和垃圾邮件的个数"""
    counts = {}  
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [8]:
#######
#示例
class_counts(training_data)
#######

{'垃圾': 4, '正常': 4}

In [9]:
def is_numeric(value):
    """看看这个值是不是数字类型"""
    return isinstance(value, int) or isinstance(value, float)

In [10]:
#######
#示例
#is_numeric(7)
is_numeric("网易")
#######

False

👇进入正题

用个套路：当我们在谈决策树的时候我们究竟在说神马

In [11]:
class Question:
    """这个class会对我们的数据进行提问

       match：如果数据是数字的话，就会采用比大小方式，如果数据是字符串的话，就会采用
       是否相等方式
       
       __repr__:把问题打印出来
    
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [19]:
## # 示例:
# 看看第一个问题
Question(1, 1)

Is 混乱 >= 1?

In [22]:
# 第二个问题
q = Question(0, '谷歌')
q

Is 域名 == 谷歌?

In [53]:
# 来来来，现在我们找个样本来测试一下
example = training_data[0]
q.match(example) # 看看我们的第一个样本的域名是不是谷歌呢
#######

True

In [24]:
def partition(rows, question):
    """到了给样本分类的时候了

    对每个样本进行提问，如果匹配就放到true的队列里，如果不匹配就放到
    false的队列里面
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [27]:
#######
# 示例
# 我们先拿网易来开刀，看看这分类的结果怎么样
true_rows, false_rows = partition(training_data, Question(0, '网易'))
# 第一行是匹配的，第二行是不匹配的
print(true_rows,'\n',false_rows)

[['网易', 2, '正常'], ['网易', 2, '垃圾'], ['网易', 1, '正常']] 
 [['谷歌', 3, '垃圾'], ['雅虎', 3, '垃圾'], ['苹果', 1, '正常'], ['雅虎', 2, '垃圾'], ['苹果', 2, '正常']]


In [28]:
true_rows, false_rows = partition(training_data, Question(1, 2))
# This will contain all the 'Red' rows.
print(true_rows,'\n',false_rows)

[['谷歌', 3, '垃圾'], ['网易', 2, '正常'], ['雅虎', 3, '垃圾'], ['网易', 2, '垃圾'], ['雅虎', 2, '垃圾'], ['苹果', 2, '正常']] 
 [['苹果', 1, '正常'], ['网易', 1, '正常']]


In [29]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [30]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [31]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [32]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [33]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [34]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.5

In [37]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, '网易'))
info_gain(true_rows, false_rows, current_uncertainty)

0.033333333333333326

In [39]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'苹果'))
info_gain(true_rows, false_rows, current_uncertainty)

0.16666666666666663

In [42]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(1,2))
info_gain(true_rows, false_rows, current_uncertainty)

0.16666666666666663

In [45]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,'苹果'))

# Here, the true_rows contain only 'Grapes'.
print(true_rows,'\n',false_rows)

[['苹果', 1, '正常'], ['苹果', 2, '正常']] 
 [['谷歌', 3, '垃圾'], ['网易', 2, '正常'], ['雅虎', 3, '垃圾'], ['网易', 2, '垃圾'], ['网易', 1, '正常'], ['雅虎', 2, '垃圾']]


In [51]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'雅虎'))
info_gain(true_rows, false_rows, current_uncertainty)

0.16666666666666669

In [47]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(1, 2))

# Here, the true_rows contain only 'Grapes'.
print(true_rows,'\n',false_rows)

[['谷歌', 3, '垃圾'], ['网易', 2, '正常'], ['雅虎', 3, '垃圾'], ['网易', 2, '垃圾'], ['雅虎', 2, '垃圾'], ['苹果', 2, '正常']] 
 [['苹果', 1, '正常'], ['网易', 1, '正常']]


In [48]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [49]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is 域名 == 雅虎?

In [52]:
best_gain

0.16666666666666669